DSC 680 Project 3

Zack DeNoto

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.linear_model import Ridge
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from lazypredict.Supervised import LazyClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense



C:\Users\Zack\AppData\Roaming\Python\Python36\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [4]:
#imports data from csv file
data = pd.read_csv('accidents.csv')

#puts data into a dataframe and prints it out
df = pd.DataFrame(data)
print(df.head())

    ID  Severity           Start_Time             End_Time  Start_Lat  \
0  A-1         2  2019-05-21 08:29:55  2019-05-21 09:29:40      34.81   
1  A-2         2  2019-10-07 17:43:09  2019-10-07 19:42:50      35.09   
2  A-3         2  2020-12-13 21:53:00  2020-12-13 22:44:00      37.15   
3  A-4         2  2018-04-17 16:51:23  2018-04-17 17:50:46      39.11   
4  A-5         3  2016-08-31 17:40:49  2016-08-31 18:10:49      26.10   

   Start_Lng  End_Lat  End_Lng  Distance(mi)  \
0     -82.27    34.81   -82.27          0.00   
1     -80.75    35.09   -80.75          0.00   
2    -121.99    37.17  -121.99          1.40   
3    -119.77    39.11  -119.77          0.00   
4     -80.27    26.10   -80.27          0.00   

                                         Description  ...  Roundabout Station  \
0            Accident on Tanner Rd at Pennbrooke Ln.  ...       False   False   
1  Accident on Houston Branch Rd at Providence Br...  ...       False   False   
2  Stationary traffic on CA-1

In [5]:
df = df.drop(columns=['ID','Zipcode', 'Description', 'Start_Lat','Start_Lng', 'End_Lat','End_Lng', 'Number','Street', 'Country','Timezone', 'Airport_Code','Weather_Timestamp', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight'])

print(df.head())

   Severity           Start_Time             End_Time  Distance(mi) Side  \
0         2  2019-05-21 08:29:55  2019-05-21 09:29:40          0.00    R   
1         2  2019-10-07 17:43:09  2019-10-07 19:42:50          0.00    R   
2         2  2020-12-13 21:53:00  2020-12-13 22:44:00          1.40    R   
3         2  2018-04-17 16:51:23  2018-04-17 17:50:46          0.00    R   
4         3  2016-08-31 17:40:49  2016-08-31 18:10:49          0.00    R   

              City       County State  Temperature(F)  Wind_Chill(F)  ...  \
0       Greenville   Greenville    SC           76.00          76.00  ...   
1        Charlotte  Mecklenburg    NC           76.00          76.00  ...   
2        Los Gatos  Santa Clara    CA           51.00          51.00  ...   
3      Carson City      Douglas    NV           53.60            nan  ...   
4  Fort Lauderdale      Broward    FL           84.20            nan  ...   

   Give_Way  Junction  No_Exit Railway  Roundabout  Station   Stop  \
0     Fals

In [6]:
orig = df

df['Give_Way'] = df['Give_Way'].astype(int)
df['Amenity'] = df['Amenity'].astype(int)
df['Bump'] = df['Bump'].astype(int)
df['Crossing'] = df['Crossing'].astype(int)
df['Junction'] = df['Junction'].astype(int)
df['No_Exit'] = df['No_Exit'].astype(int)
df['Railway'] = df['Railway'].astype(int)
df['Roundabout'] = df['Roundabout'].astype(int)
df['Station'] = df['Station'].astype(int)
df['Stop'] = df['Stop'].astype(int)
df['Traffic_Calming'] = df['Traffic_Calming'].astype(int)
df['Traffic_Signal'] = df['Traffic_Signal'].astype(int)
df['Turning_Loop'] = df['Turning_Loop'].astype(int)



print(df.head())

   Severity           Start_Time             End_Time  Distance(mi) Side  \
0         2  2019-05-21 08:29:55  2019-05-21 09:29:40          0.00    R   
1         2  2019-10-07 17:43:09  2019-10-07 19:42:50          0.00    R   
2         2  2020-12-13 21:53:00  2020-12-13 22:44:00          1.40    R   
3         2  2018-04-17 16:51:23  2018-04-17 17:50:46          0.00    R   
4         3  2016-08-31 17:40:49  2016-08-31 18:10:49          0.00    R   

              City       County State  Temperature(F)  Wind_Chill(F)  ...  \
0       Greenville   Greenville    SC           76.00          76.00  ...   
1        Charlotte  Mecklenburg    NC           76.00          76.00  ...   
2        Los Gatos  Santa Clara    CA           51.00          51.00  ...   
3      Carson City      Douglas    NV           53.60            nan  ...   
4  Fort Lauderdale      Broward    FL           84.20            nan  ...   

   Give_Way  Junction  No_Exit Railway  Roundabout  Station Stop  \
0         0 

In [7]:
df['Start_Time'] = pd.to_datetime(df['Start_Time'], format='%Y-%m-%d %H:%M:%S')
df['End_Time'] = pd.to_datetime(df['End_Time'], format='%Y-%m-%d %H:%M:%S')

df['time'] = df['End_Time'] - df['Start_Time']

print(df['time'].head())

0   00:59:45
1   01:59:41
2   00:51:00
3   00:59:23
4   00:30:00
Name: time, dtype: timedelta64[ns]


In [8]:
dnc = df

dnc = dnc.drop(columns=['City','County', 'State', 'Side'])

for col in dnc.columns:
    print(col)

Severity
Start_Time
End_Time
Distance(mi)
Temperature(F)
Wind_Chill(F)
Humidity(%)
Pressure(in)
Visibility(mi)
Wind_Direction
Wind_Speed(mph)
Precipitation(in)
Weather_Condition
Amenity
Bump
Crossing
Give_Way
Junction
No_Exit
Railway
Roundabout
Station
Stop
Traffic_Calming
Traffic_Signal
Turning_Loop
time


In [9]:
is_NaN = dnc.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = dnc[row_has_NaN]

print(rows_with_NaN)


         Severity          Start_Time            End_Time  Distance(mi)  \
3               2 2018-04-17 16:51:23 2018-04-17 17:50:46          0.00   
4               3 2016-08-31 17:40:49 2016-08-31 18:10:49          0.00   
5               3 2018-10-17 16:40:36 2018-10-17 17:10:18          0.00   
8               2 2018-05-23 16:50:24 2018-05-23 22:50:24          0.97   
12              2 2018-04-19 12:48:57 2018-04-19 18:48:57          0.62   
...           ...                 ...                 ...           ...   
2906601         3 2018-03-12 08:24:41 2018-03-12 08:52:48          0.00   
2906603         2 2016-12-19 06:34:56 2016-12-19 07:19:45          0.01   
2906605         2 2018-06-28 08:49:01 2018-06-28 09:18:51          0.00   
2906606         2 2019-01-10 02:01:01 2019-01-10 02:30:32          0.00   
2906609         2 2018-03-26 13:35:03 2018-03-26 14:39:00          0.00   

         Temperature(F)  Wind_Chill(F)  Humidity(%)  Pressure(in)  \
3                 53.60       

In [10]:
dnc['Wind_Chill(F)'] = dnc['Wind_Chill(F)'].fillna(0)

is_NaN = dnc.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = dnc[row_has_NaN]

print(rows_with_NaN)

         Severity          Start_Time            End_Time  Distance(mi)  \
3               2 2018-04-17 16:51:23 2018-04-17 17:50:46          0.00   
4               3 2016-08-31 17:40:49 2016-08-31 18:10:49          0.00   
5               3 2018-10-17 16:40:36 2018-10-17 17:10:18          0.00   
8               2 2018-05-23 16:50:24 2018-05-23 22:50:24          0.97   
12              2 2018-04-19 12:48:57 2018-04-19 18:48:57          0.62   
...           ...                 ...                 ...           ...   
2906601         3 2018-03-12 08:24:41 2018-03-12 08:52:48          0.00   
2906603         2 2016-12-19 06:34:56 2016-12-19 07:19:45          0.01   
2906605         2 2018-06-28 08:49:01 2018-06-28 09:18:51          0.00   
2906606         2 2019-01-10 02:01:01 2019-01-10 02:30:32          0.00   
2906609         2 2018-03-26 13:35:03 2018-03-26 14:39:00          0.00   

         Temperature(F)  Wind_Chill(F)  Humidity(%)  Pressure(in)  \
3                 53.60       

In [11]:
for col in dnc.columns:
    print(col +': ' + str(dnc[col].isna().sum()))

Severity: 0
Start_Time: 0
End_Time: 0
Distance(mi): 0
Temperature(F): 67224
Wind_Chill(F): 0
Humidity(%): 71270
Pressure(in): 56908
Visibility(mi): 72078
Wind_Direction: 63474
Wind_Speed(mph): 307163
Precipitation(in): 1301326
Weather_Condition: 71851
Amenity: 0
Bump: 0
Crossing: 0
Give_Way: 0
Junction: 0
No_Exit: 0
Railway: 0
Roundabout: 0
Station: 0
Stop: 0
Traffic_Calming: 0
Traffic_Signal: 0
Turning_Loop: 0
time: 0


In [12]:
dnc['Humidity(%)'] = dnc['Humidity(%)'].fillna(0)
dnc['Visibility(mi)'] = dnc['Visibility(mi)'].fillna(0)
dnc['Wind_Speed(mph)'] = dnc['Wind_Speed(mph)'].fillna(0)
dnc['Precipitation(in)'] = dnc['Precipitation(in)'].fillna(0)
dnc['Humidity(%)'] = dnc['Humidity(%)'].fillna(0)

dnc = dnc.drop(columns=['Wind_Direction','Weather_Condition'])

for col in dnc.columns:
    print(col +': ' + str(dnc[col].isna().sum()))

Severity: 0
Start_Time: 0
End_Time: 0
Distance(mi): 0
Temperature(F): 67224
Wind_Chill(F): 0
Humidity(%): 0
Pressure(in): 56908
Visibility(mi): 0
Wind_Speed(mph): 0
Precipitation(in): 0
Amenity: 0
Bump: 0
Crossing: 0
Give_Way: 0
Junction: 0
No_Exit: 0
Railway: 0
Roundabout: 0
Station: 0
Stop: 0
Traffic_Calming: 0
Traffic_Signal: 0
Turning_Loop: 0
time: 0


In [13]:
dnc['Pressure(in)'] = dnc['Pressure(in)'].fillna(0)
dnc = dnc.dropna()

for col in dnc.columns:
    print(col +': ' + str(dnc[col].isna().sum()))

Severity: 0
Start_Time: 0
End_Time: 0
Distance(mi): 0
Temperature(F): 0
Wind_Chill(F): 0
Humidity(%): 0
Pressure(in): 0
Visibility(mi): 0
Wind_Speed(mph): 0
Precipitation(in): 0
Amenity: 0
Bump: 0
Crossing: 0
Give_Way: 0
Junction: 0
No_Exit: 0
Railway: 0
Roundabout: 0
Station: 0
Stop: 0
Traffic_Calming: 0
Traffic_Signal: 0
Turning_Loop: 0
time: 0


In [14]:
noncovdnc = dnc[dnc['Start_Time'] < '2020-03-01']
print(noncovdnc)

covdnc = dnc[dnc['Start_Time'] >= '2020-03-01']
print(covdnc)

         Severity          Start_Time            End_Time  Distance(mi)  \
0               2 2019-05-21 08:29:55 2019-05-21 09:29:40          0.00   
1               2 2019-10-07 17:43:09 2019-10-07 19:42:50          0.00   
3               2 2018-04-17 16:51:23 2018-04-17 17:50:46          0.00   
4               3 2016-08-31 17:40:49 2016-08-31 18:10:49          0.00   
5               3 2018-10-17 16:40:36 2018-10-17 17:10:18          0.00   
...           ...                 ...                 ...           ...   
2906603         2 2016-12-19 06:34:56 2016-12-19 07:19:45          0.01   
2906605         2 2018-06-28 08:49:01 2018-06-28 09:18:51          0.00   
2906606         2 2019-01-10 02:01:01 2019-01-10 02:30:32          0.00   
2906608         2 2019-12-29 22:38:00 2019-12-29 23:38:00          0.00   
2906609         2 2018-03-26 13:35:03 2018-03-26 14:39:00          0.00   

         Temperature(F)  Wind_Chill(F)  Humidity(%)  Pressure(in)  \
0                 76.00       

In [15]:
print(dnc['time'].mean())
print(covdnc['time'].mean())
print(noncovdnc['time'].mean())

0 days 02:50:54.495941
0 days 03:10:46.475515
0 days 02:42:01.252826


In [16]:
print(dnc['Severity'].mean())
print(covdnc['Severity'].mean())
print(noncovdnc['Severity'].mean())

2.2887835609529668
2.1351787114182055
2.357500114692344


In [17]:
dnc['time(in hours)'] = (dnc['time'] / np.timedelta64(1,'m')).astype(int)
dnc['time(in hours)'] = round(dnc['time(in hours)']/60,2)

covdnc['time(in hours)'] = (covdnc['time'] / np.timedelta64(1,'m')).astype(int)
covdnc['time(in hours)'] = round(covdnc['time(in hours)']/60,2)

noncovdnc['time(in hours)'] = (noncovdnc['time'] / np.timedelta64(1,'m')).astype(int)
noncovdnc['time(in hours)'] = round(noncovdnc['time(in hours)']/60,2)




covdnc = covdnc.drop(columns=['time'])
noncovdnc = dnc.drop(columns=['time'])
dnc = dnc.drop(columns=['time'])

print(dnc)
print(covdnc)
print(noncovdnc)


         Severity          Start_Time            End_Time  Distance(mi)  \
0               2 2019-05-21 08:29:55 2019-05-21 09:29:40          0.00   
1               2 2019-10-07 17:43:09 2019-10-07 19:42:50          0.00   
2               2 2020-12-13 21:53:00 2020-12-13 22:44:00          1.40   
3               2 2018-04-17 16:51:23 2018-04-17 17:50:46          0.00   
4               3 2016-08-31 17:40:49 2016-08-31 18:10:49          0.00   
...           ...                 ...                 ...           ...   
2906605         2 2018-06-28 08:49:01 2018-06-28 09:18:51          0.00   
2906606         2 2019-01-10 02:01:01 2019-01-10 02:30:32          0.00   
2906607         2 2020-11-23 12:52:00 2020-11-24 00:47:37          0.11   
2906608         2 2019-12-29 22:38:00 2019-12-29 23:38:00          0.00   
2906609         2 2018-03-26 13:35:03 2018-03-26 14:39:00          0.00   

         Temperature(F)  Wind_Chill(F)  Humidity(%)  Pressure(in)  \
0                 76.00       

In [18]:
covdnc = covdnc.drop(columns=['Start_Time','End_Time'])
noncovdnc = noncovdnc.drop(columns=['Start_Time','End_Time'])
dnc = dnc.drop(columns=['Start_Time','End_Time'])

print(dnc.head())
print(covdnc.head())
print(noncovdnc.head())

   Severity  Distance(mi)  Temperature(F)  Wind_Chill(F)  Humidity(%)  \
0         2          0.00           76.00          76.00        52.00   
1         2          0.00           76.00          76.00        62.00   
2         2          1.40           51.00          51.00        80.00   
3         2          0.00           53.60           0.00        16.00   
4         3          0.00           84.20           0.00        84.00   

   Pressure(in)  Visibility(mi)  Wind_Speed(mph)  Precipitation(in)  Amenity  \
0         28.91           10.00             7.00               0.00        0   
1         29.30           10.00             3.00               0.00        0   
2         30.17           10.00             6.00               0.00        0   
3         30.16           10.00             4.60               0.00        0   
4         29.92           10.00            13.80               0.00        0   

   ...  Junction  No_Exit  Railway  Roundabout  Station  Stop  \
0  ...         

In [19]:
covdnc_samp = covdnc.sample(n=50000)
print(covdnc_samp)

         Severity  Distance(mi)  Temperature(F)  Wind_Chill(F)  Humidity(%)  \
958901          2          0.09           48.00          48.00        57.00   
2250300         2          0.44           65.00          65.00        97.00   
1466125         2          0.05           46.00          44.00        76.00   
1574639         2          0.20           44.00          44.00        79.00   
1731000         3          0.00           66.00          66.00        20.00   
...           ...           ...             ...            ...          ...   
1317917         3          0.47           54.00          54.00         0.00   
885464          3          0.00           56.00          56.00        34.00   
1335874         2          0.06           35.00          28.00        96.00   
1156790         2          0.10           77.00          77.00        79.00   
1242882         2          0.87           55.00          55.00        94.00   

         Pressure(in)  Visibility(mi)  Wind_Speed(m

In [20]:
Xcov = covdnc_samp.drop(['Severity'], axis=1)

ycov= covdnc_samp.Severity


X_traincov, X_testcov, y_traincov, y_testcov = train_test_split(Xcov, ycov)


In [21]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)

models, predictions = clf.fit(X_traincov, X_testcov, y_traincov, y_testcov)
models


100%|██████████████████████████████████████████| 29/29 [03:19<00:00,  6.88s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
DecisionTreeClassifier,0.78,0.44,None,0.78,0.26
NearestCentroid,0.41,0.42,None,0.50,0.06
BaggingClassifier,0.83,0.42,None,0.80,1.43
XGBClassifier,0.84,0.40,None,0.81,10.51
LGBMClassifier,0.84,0.38,None,0.80,0.75
RandomForestClassifier,0.84,0.37,None,0.80,3.75
ExtraTreeClassifier,0.75,0.36,None,0.75,0.08
PassiveAggressiveClassifier,0.70,0.34,None,0.72,0.26
ExtraTreesClassifier,0.83,0.33,None,0.79,3.06


In [22]:
rf = RandomForestClassifier()
rf.fit(X_traincov, y_traincov)
y_predcov = rf.predict(X_testcov)

print('Random Forest Evaluation Metrics')
print('Accuracy: ' + str(metrics.accuracy_score(y_testcov,y_predcov)))
print('F1 Score: ' + str(metrics.f1_score(y_testcov,y_pred, average='macro')))

Random Forest Evaluation Metrics
Accuracy: 0.84256
F1 Score: 0.4107862093831272


In [29]:
lr = LogisticRegression()
lr.fit(X_traincov, y_traincov)
y_pred_lrcov = lr.predict(X_testcov)

print('Logistic Regresstion Evaluation Metrics')
print('Accuracy: ' + str(metrics.accuracy_score(y_testcov,y_pred_lrcov)))
print('F1 Score: ' + str(metrics.f1_score(y_testcov,y_pred_lrcov, average='macro')))

Logistic Regresstion Evaluation Metrics
Accuracy: 0.83048
F1 Score: 0.22864837966940632


In [30]:
xgb = XGBClassifier()
xgb.fit(X_traincov, y_traincov)
y_pred_xgbcov = xgb.predict(X_testcov)

print('XGBoost Evaluation Metrics')
print('Accuracy: ' + str(metrics.accuracy_score(y_testcov,y_pred_xgbcov)))
print('F1 Score: ' + str(metrics.f1_score(y_testcov,y_pred_xgbcov, average='macro')))

XGBoost Evaluation Metrics
Accuracy: 0.8444
F1 Score: 0.45519915524430016


In [31]:
sgd = SGDClassifier()
sgd.fit(X_traincov, y_traincov)
y_pred_sgdcov = sgd.predict(X_testcov)

print('Stochastic Gradient Descent Evaluation Metrics')
print('Accuracy: ' + str(metrics.accuracy_score(y_testcov,y_pred_sgdcov)))
print('F1 Score: ' + str(metrics.f1_score(y_testcov,y_pred_sgdcov, average='macro')))

Stochastic Gradient Descent Evaluation Metrics
Accuracy: 0.83112
F1 Score: 0.22696290470572814


In [32]:
knn = KNeighborsClassifier()
knn.fit(X_traincov, y_traincov)
y_pred_knncov = knn.predict(X_testcov)

print('K-Nearest Neighbor Evaluation Metrics')
print('Accuracy: ' + str(metrics.accuracy_score(y_testcov,y_pred_knncov)))
print('F1 Score: ' + str(metrics.f1_score(y_testcov,y_pred_knncov, average='macro')))

K-Nearest Neighbor Evaluation Metrics
Accuracy: 0.81376
F1 Score: 0.2985102777867997


In [23]:
noncovdnc_sample = noncovdnc.sample(n=50000)
print(noncovdnc_sample)

         Severity  Distance(mi)  Temperature(F)  Wind_Chill(F)  Humidity(%)  \
1586038         3          0.00           78.00          78.00        74.00   
893224          2          0.00           53.10           0.00        66.00   
751425          2          3.10           64.00          64.00        87.00   
208833          2          0.80           44.10          36.80        38.00   
1429590         2          0.78           90.00          90.00        10.00   
...           ...           ...             ...            ...          ...   
2237628         2          0.00           84.00          84.00        76.00   
1625639         2          0.05           58.00          58.00        90.00   
2132024         2          0.01           37.00           0.00        93.00   
2353234         2          0.00           68.00           0.00        32.00   
1661689         3          0.00           56.00          56.00        72.00   

         Pressure(in)  Visibility(mi)  Wind_Speed(m

In [24]:
Xnon = noncovdnc_sample.drop(['Severity'], axis=1)

ynon= noncovdnc_sample.Severity


X_trainnon, X_testnon, y_trainnon, y_testnon = train_test_split(Xnon, ynon)


In [25]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)

models, predictions = clf.fit(X_trainnon, X_testnon, y_trainnon, y_testnon)
models

100%|██████████████████████████████████████████| 29/29 [03:19<00:00,  6.89s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
NearestCentroid,0.41,0.45,None,0.47,0.06
GaussianNB,0.05,0.40,None,0.01,0.07
DecisionTreeClassifier,0.65,0.39,None,0.65,0.25
BaggingClassifier,0.71,0.39,None,0.68,1.35
PassiveAggressiveClassifier,0.54,0.38,None,0.58,0.14
XGBClassifier,0.74,0.37,None,0.68,3.30
LGBMClassifier,0.74,0.36,None,0.68,0.74
RandomForestClassifier,0.73,0.36,None,0.68,3.73
ExtraTreeClassifier,0.63,0.34,None,0.63,0.09


In [35]:
rf = RandomForestClassifier()
rf.fit(X_trainnon, y_trainnon)
y_prednon = rf.predict(X_testnon)

print('Random Forest Evaluation Metrics')
print('Accuracy: ' + str(metrics.accuracy_score(y_testnon,y_prednon)))
print('F1 Score: ' + str(metrics.f1_score(y_testnon,y_prednon, average='macro')))

Random Forest Evaluation Metrics
Accuracy: 0.73176
F1 Score: 0.38886400466250665


In [33]:
lr = LogisticRegression()
lr.fit(X_trainnon, y_trainnon)
y_pred_lrnon = lr.predict(X_testnon)

print('Logistic Regresstion Evaluation Metrics')
print('Accuracy: ' + str(metrics.accuracy_score(y_testnon,y_pred_lrnon)))
print('F1 Score: ' + str(metrics.f1_score(y_testnon,y_pred_lrnon, average='macro')))

Logistic Regresstion Evaluation Metrics
Accuracy: 0.72976
F1 Score: 0.22976321520700513


In [36]:
xgb = XGBClassifier()
xgb.fit(X_trainnon, y_trainnon)
y_pred_xgbnon = xgb.predict(X_testnon)

print('XGBoost Evaluation Metrics')
print('Accuracy: ' + str(metrics.accuracy_score(y_testnon,y_pred_xgbnon)))
print('F1 Score: ' + str(metrics.f1_score(y_testnon,y_pred_xgbnon, average='macro')))

XGBoost Evaluation Metrics
Accuracy: 0.7372
F1 Score: 0.39764315939773376


In [37]:
sgd = SGDClassifier()
sgd.fit(X_trainnon, y_trainnon)
y_pred_sgdnon = sgd.predict(X_testnon)

print('Stochastic Gradient Descent Evaluation Metrics')
print('Accuracy: ' + str(metrics.accuracy_score(y_testnon,y_pred_sgdnon)))
print('F1 Score: ' + str(metrics.f1_score(y_testnon,y_pred_sgdnon, average='macro')))

Stochastic Gradient Descent Evaluation Metrics
Accuracy: 0.69608
F1 Score: 0.2885638532185745


In [38]:
knn = KNeighborsClassifier()
knn.fit(X_trainnon, y_trainnon)
y_pred_knnnon = knn.predict(X_testnon)

print('K-Nearest Neighbor Evaluation Metrics')
print('Accuracy: ' + str(metrics.accuracy_score(y_testnon,y_pred_knnnon)))
print('F1 Score: ' + str(metrics.f1_score(y_testnon,y_pred_knnnon, average='macro')))

K-Nearest Neighbor Evaluation Metrics
Accuracy: 0.69184
F1 Score: 0.2719291574162853


In [26]:
dnc_samp = dnc.sample(n=50000)
print(dnc_samp)

         Severity  Distance(mi)  Temperature(F)  Wind_Chill(F)  Humidity(%)  \
1898817         2          0.00           72.00          72.00        38.00   
284308          2          0.00           60.00          60.00        96.00   
262865          2          0.87           28.90          16.10        56.00   
1240023         2          0.09           51.10           0.00        80.00   
2231734         2          0.00           39.90           0.00        58.00   
...           ...           ...             ...            ...          ...   
2081468         3          0.57           86.00           0.00        37.00   
1699062         3          0.00           76.00          76.00        31.00   
2510998         2          0.14           63.00          63.00        81.00   
2220083         2          0.00           57.90           0.00        90.00   
1958819         2          0.73           60.00          60.00        80.00   

         Pressure(in)  Visibility(mi)  Wind_Speed(m

In [27]:
X = dnc_samp.drop(['Severity'], axis=1)

y= dnc_samp.Severity


X_train, X_test, y_train, y_test = train_test_split(X, y)


In [28]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)

models, predictions = clf.fit(X_train, X_test, y_train, y_test)
models

100%|██████████████████████████████████████████| 29/29 [03:17<00:00,  6.82s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
NearestCentroid,0.40,0.48,None,0.46,0.06
DecisionTreeClassifier,0.66,0.44,None,0.66,0.25
BaggingClassifier,0.72,0.42,None,0.69,1.34
XGBClassifier,0.74,0.40,None,0.69,3.28
LGBMClassifier,0.74,0.39,None,0.68,0.70
RandomForestClassifier,0.74,0.37,None,0.69,3.72
ExtraTreeClassifier,0.64,0.35,None,0.64,0.09
ExtraTreesClassifier,0.73,0.34,None,0.68,3.47
Perceptron,0.66,0.33,None,0.63,0.15


In [39]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print('Random Forest Evaluation Metrics')
print('Accuracy: ' + str(metrics.accuracy_score(y_test,y_pred)))
print('F1 Score: ' + str(metrics.f1_score(y_test,y_pred, average='macro')))

Random Forest Evaluation Metrics
Accuracy: 0.73296
F1 Score: 0.4058284767733066


In [40]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

print('Logistic Regresstion Evaluation Metrics')
print('Accuracy: ' + str(metrics.accuracy_score(y_test,y_pred_lr)))
print('F1 Score: ' + str(metrics.f1_score(y_test,y_pred_lr, average='macro')))

Logistic Regresstion Evaluation Metrics
Accuracy: 0.73408
F1 Score: 0.22262972213501528


In [41]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

print('XGBoost Evaluation Metrics')
print('Accuracy: ' + str(metrics.accuracy_score(y_test,y_pred_xgb)))
print('F1 Score: ' + str(metrics.f1_score(y_test,y_pred_xgb, average='macro')))

XGBoost Evaluation Metrics
Accuracy: 0.73824
F1 Score: 0.43657713002204995


In [42]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
y_pred_sgd = sgd.predict(X_test)

print('Stochastic Gradient Descent Evaluation Metrics')
print('Accuracy: ' + str(metrics.accuracy_score(y_test,y_pred_sgd)))
print('F1 Score: ' + str(metrics.f1_score(y_test,y_pred_sgd, average='macro')))

Stochastic Gradient Descent Evaluation Metrics
Accuracy: 0.59816
F1 Score: 0.23843778780891128


In [43]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

print('K-Nearest Neighbor Evaluation Metrics')
print('Accuracy: ' + str(metrics.accuracy_score(y_test,y_pred_knn)))
print('F1 Score: ' + str(metrics.f1_score(y_test,y_pred_knn, average='macro')))

K-Nearest Neighbor Evaluation Metrics
Accuracy: 0.69312
F1 Score: 0.2953830602190587
